# IMPORT AND CHECK DATA

In [1]:
import pandas as pd 
raw_path = "data/raw/" 
click = pd.read_csv(raw_path+"feature_clickstream.csv") 
attr = pd.read_csv(raw_path+"features_attributes.csv") 
fin = pd.read_csv(raw_path+"features_financials.csv") 
lms = pd.read_csv(raw_path+"lms_loan_daily.csv")

In [26]:
for name, df in {"click":click,"attr":attr,"fin":fin,"lms":lms}.items(): 
    print(f"\n== {name} ==") 
    display(df.info()) 


== click ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215376 entries, 0 to 215375
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   fe_1           215376 non-null  int64 
 1   fe_2           215376 non-null  int64 
 2   fe_3           215376 non-null  int64 
 3   fe_4           215376 non-null  int64 
 4   fe_5           215376 non-null  int64 
 5   fe_6           215376 non-null  int64 
 6   fe_7           215376 non-null  int64 
 7   fe_8           215376 non-null  int64 
 8   fe_9           215376 non-null  int64 
 9   fe_10          215376 non-null  int64 
 10  fe_11          215376 non-null  int64 
 11  fe_12          215376 non-null  int64 
 12  fe_13          215376 non-null  int64 
 13  fe_14          215376 non-null  int64 
 14  fe_15          215376 non-null  int64 
 15  fe_16          215376 non-null  int64 
 16  fe_17          215376 non-null  int64 
 17  fe_18          215376 non-null  int

None


== attr ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12500 entries, 0 to 12499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer_ID    12500 non-null  object
 1   Name           12500 non-null  object
 2   Age            12500 non-null  object
 3   SSN            12500 non-null  object
 4   Occupation     12500 non-null  object
 5   snapshot_date  12500 non-null  object
dtypes: object(6)
memory usage: 586.1+ KB


None


== fin ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12500 entries, 0 to 12499
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               12500 non-null  object 
 1   Annual_Income             12500 non-null  object 
 2   Monthly_Inhand_Salary     12500 non-null  float64
 3   Num_Bank_Accounts         12500 non-null  int64  
 4   Num_Credit_Card           12500 non-null  int64  
 5   Interest_Rate             12500 non-null  int64  
 6   Num_of_Loan               12500 non-null  object 
 7   Type_of_Loan              11074 non-null  object 
 8   Delay_from_due_date       12500 non-null  int64  
 9   Num_of_Delayed_Payment    12500 non-null  object 
 10  Changed_Credit_Limit      12500 non-null  object 
 11  Num_Credit_Inquiries      12500 non-null  float64
 12  Credit_Mix                12500 non-null  object 
 13  Outstanding_Debt          12500 non-null  object 


None


== lms ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137500 entries, 0 to 137499
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   loan_id          137500 non-null  object 
 1   Customer_ID      137500 non-null  object 
 2   loan_start_date  137500 non-null  object 
 3   tenure           137500 non-null  int64  
 4   installment_num  137500 non-null  int64  
 5   loan_amt         137500 non-null  int64  
 6   due_amt          137500 non-null  float64
 7   paid_amt         137500 non-null  float64
 8   overdue_amt      137500 non-null  float64
 9   balance          137500 non-null  float64
 10  snapshot_date    137500 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 11.5+ MB


None

In [27]:
print(click["Customer_ID"].nunique(), click.shape[0]) 
print(lms[["loan_id","snapshot_date"]].duplicated().sum())

8974 215376
0


In [28]:
loan_view = lms.merge(attr, on="Customer_ID", how="left") 
print(loan_view.isna().mean().head())

loan_id            0.0
Customer_ID        0.0
loan_start_date    0.0
tenure             0.0
installment_num    0.0
dtype: float64


# BRONZE TABLE PROCESSING

In [2]:
import pyspark
import os
from pyspark.sql import SparkSession, functions as F

In [30]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("assignment1-bronze")
        .master("local[*]")
        .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
print("Spark ready ✅  — version:", spark.version)

Spark ready ✅  — version: 3.5.1


In [31]:
from pathlib import Path

RAW_DIR      = Path("data/raw")           
BRONZE_ROOT  = Path("data_mart/bronze")   
BRONZE_ROOT.mkdir(parents=True, exist_ok=True)

CSV_SOURCES = {
    "feature_clickstream"  : "feature_clickstream.csv",
    "features_attributes"  : "features_attributes.csv",
    "features_financials"  : "features_financials.csv",
    "lms_loan_daily"       : "lms_loan_daily.csv",
}
PARTITION_COL = "snapshot_date"

In [38]:
from datetime import datetime
df_dates = (
    spark.read.option("header", "true").csv(str(RAW_DIR / CSV_SOURCES["lms_loan_daily"]))
         .select(F.min("snapshot_date").alias("min_dt"),
                 F.max("snapshot_date").alias("max_dt"))
         .collect()[0]
)

start_dt = datetime.strptime(df_dates["min_dt"], "%Y-%m-%d")
end_dt   = datetime.strptime(df_dates["max_dt"], "%Y-%m-%d")

start_date_str = start_dt.strftime("%Y-%m-01")
end_date_str   = end_dt.replace(day=1).strftime("%Y-%m-%d")

def generate_first_of_month_dates(start_str, end_str):
    start = datetime.strptime(start_str, "%Y-%m-%d")
    end   = datetime.strptime(end_str,   "%Y-%m-%d")
    dates = []
    cur = datetime(start.year, start.month, 1)
    while cur <= end:
        dates.append(cur.strftime("%Y-%m-%d"))
        cur = datetime(cur.year + (cur.month // 12),
                       (cur.month % 12) + 1, 1)
    return dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
print("Back-fill months:", dates_str_lst[:3], "...", dates_str_lst[-1])

for table, raw_csv in CSV_SOURCES.items():
    src_path = RAW_DIR / raw_csv
    bronze_dir = BRONZE_ROOT / table
    bronze_dir.mkdir(parents=True, exist_ok=True)

    # read full table once
    sdf_full = (
        spark.read.option("header", "true").option("inferSchema", "true")
             .csv(str(src_path))
    )

    for ds in dates_str_lst:
        snap_dt = datetime.strptime(ds, "%Y-%m-%d")

        slice_df = sdf_full.filter(F.col("snapshot_date") == snap_dt)

        if slice_df.rdd.isEmpty():
            print(f"[{table}] {ds} – 0 rows (skip)")
            continue

        fname = f"bronze_{table}_{ds.replace('-', '_')}.csv"
        out_path = bronze_dir / fname

        # overwrite if file exists
        slice_df.toPandas().to_csv(out_path, index=False)
        print(f"[{table}] {ds}  → wrote {out_path}")

print("\nBronze CSV partitions complete ✅")
spark.stop()

Back-fill months: ['2023-01-01', '2023-02-01', '2023-03-01'] ... 2025-11-01


[feature_clickstream] 2023-01-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_01_01.csv
[feature_clickstream] 2023-02-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_02_01.csv
[feature_clickstream] 2023-03-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_03_01.csv
[feature_clickstream] 2023-04-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_04_01.csv


[feature_clickstream] 2023-05-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_05_01.csv


[feature_clickstream] 2023-06-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_06_01.csv
[feature_clickstream] 2023-07-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_07_01.csv
[feature_clickstream] 2023-08-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_08_01.csv


[feature_clickstream] 2023-09-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_09_01.csv
[feature_clickstream] 2023-10-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_10_01.csv
[feature_clickstream] 2023-11-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_11_01.csv
[feature_clickstream] 2023-12-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2023_12_01.csv
[feature_clickstream] 2024-01-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2024_01_01.csv
[feature_clickstream] 2024-02-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2024_02_01.csv
[feature_clickstream] 2024-03-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2024_03_01.csv
[feature_clickstream] 2024-04-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2024_04_01.csv
[feature_clickstream] 2024-05-01

...

[feature_clickstream] 2024-12-01  → wrote data_mart/bronze/feature_clickstream/bronze_feature_clickstream_2024_12_01.csv


[feature_clickstream] 2025-01-01 – 0 rows (skip)
[feature_clickstream] 2025-02-01 – 0 rows (skip)
[feature_clickstream] 2025-03-01 – 0 rows (skip)
[feature_clickstream] 2025-04-01 – 0 rows (skip)
[feature_clickstream] 2025-05-01 – 0 rows (skip)


[feature_clickstream] 2025-06-01 – 0 rows (skip)


[feature_clickstream] 2025-07-01 – 0 rows (skip)
[feature_clickstream] 2025-08-01 – 0 rows (skip)


[feature_clickstream] 2025-09-01 – 0 rows (skip)
[feature_clickstream] 2025-10-01 – 0 rows (skip)
[feature_clickstream] 2025-11-01 – 0 rows (skip)
[features_attributes] 2023-01-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_01_01.csv
[features_attributes] 2023-02-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_02_01.csv
[features_attributes] 2023-03-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_03_01.csv
[features_attributes] 2023-04-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_04_01.csv
[features_attributes] 2023-05-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_05_01.csv
[features_attributes] 2023-06-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_06_01.csv
[features_attributes] 2023-07-01  → wrote data_mart/bronze/features_attributes/bronze_features_attributes_2023_07_01.csv
[featu

# SILVER TABLE PROCESSING 

In [3]:
from typing import Dict, Iterable
from pathlib import Path
from datetime import datetime
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import (
    StringType, IntegerType, FloatType, DateType, TimestampType)

In [4]:
spark = (SparkSession.builder
         .appName("silver-notebook-test")
         .master("local[*]")       
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 02:43:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
BRONZE_ROOT = Path("data_mart/bronze")     
SILVER_ROOT = Path("data_mart/silver_test")  
PARTITION_COL = "snapshot_date"

In [6]:
def _cast_columns(df, type_map: Dict[str, object]):
    for col_name, spark_type in type_map.items():
        if col_name in df.columns:
            df = df.withColumn(col_name, F.col(col_name).cast(spark_type))
    return df

In [39]:
Y_RE = r"(\d+)\s*Years?"
M_RE = r"(\d+)\s*Months?"

def credit_age_to_months(col_name: str = "Credit_History_Age") -> F.Column:
    yrs = F.regexp_extract(F.col(col_name), Y_RE, 1).cast(IntegerType())
    mth = F.regexp_extract(F.col(col_name), M_RE, 1).cast(IntegerType())
    return (yrs * 12 + mth).cast(IntegerType())

In [8]:
def augment_lms(df):
    df = (
        df.withColumn("mob", F.col("installment_num").cast(IntegerType()))
          .withColumn(
              "installments_missed",
              F.ceil(F.col("overdue_amt") / F.col("due_amt")).cast(IntegerType())
          )
          .fillna(0, subset=["installments_missed"])
          .withColumn(
              "first_missed_date",
              F.when(
                  F.col("installments_missed") > 0,
                  F.add_months(F.col("snapshot_date"), -1 * F.col("installments_missed"))
              ).cast(DateType())
          )
          .withColumn(
              "dpd",
              F.when(F.col("overdue_amt") > 0,
                     F.datediff(F.col("snapshot_date"), F.col("first_missed_date")))
               .otherwise(0).cast(IntegerType())
          )
          .withColumn("remaining_term", F.col("tenure") - F.col("installment_num"))
          .withColumn("days_since_origination",
                      F.datediff(F.col("snapshot_date"), F.col("loan_start_date")))
    )
    return df

In [23]:
def augment_clickstream(df):
    feat_cols = [f"fe_{i}" for i in range(1, 21)]
    
    # Cast all clickstream features to IntegerType
    for c in feat_cols:
        df = df.withColumn(c, F.col(c).cast(IntegerType()))
    
    # Compute mean of clickstream features
    df = df.withColumn(
        "clickstream_mean",
        sum([F.col(c) for c in feat_cols]) / len(feat_cols)
    )
    
    return df

In [16]:
def augment_attributes(df):
    df = (df.withColumn("Age", F.regexp_replace("Age", r"_$", "").cast(IntegerType()))
            .withColumn("Occupation", F.when(F.col("Occupation").isin("_______", "", None), "NA")
                                       .otherwise(F.col("Occupation")))
         )

    df = df.withColumn("Age", F.when((F.col("Age") < 0) | (F.col("Age") > 100), None)
                                 .otherwise(F.col("Age")))

    df = (df
          .withColumn("age_band",
              F.when(F.col("Age") < 25, "18-24")
               .when(F.col("Age") < 35, "25-34")
               .when(F.col("Age") < 45, "35-44")
               .when(F.col("Age") < 55, "45-54")
               .otherwise("55+"))
          .withColumn("has_valid_ssn",
              F.when(F.col("SSN").rlike(r"^\d{3}-\d{2}-\d{4}$"), 1).otherwise(0))
         )
    return df

In [36]:
def augment_financials(df):
    df = (
        df
        .withColumn("Annual_Income",
            F.regexp_replace("Annual_Income", r"_$", "").cast(FloatType()))
        .withColumn("Outstanding_Debt",
            F.regexp_replace("Outstanding_Debt", r"_$", "").cast(FloatType()))
        .withColumn("Amount_invested_monthly",
            F.regexp_replace("Amount_invested_monthly", r"^_+|_+$", "").cast(FloatType()))
        .withColumn("Changed_Credit_Limit",
            F.regexp_replace("Changed_Credit_Limit", r"_$", "").cast(FloatType()))
        .withColumn("Credit_Mix",
            F.when(F.col("Credit_Mix").contains("_"), None)
             .otherwise(F.col("Credit_Mix")))
        .withColumn("Num_of_Loan",
            F.regexp_replace("Num_of_Loan", r"_$", "").cast(IntegerType()))
        .withColumn("Num_of_Delayed_Payment",
            F.regexp_replace("Num_of_Delayed_Payment", r"_$", "").cast(IntegerType()))
    )

    df = (
        df
        .withColumn("Num_of_Loan",
            F.when(F.col("Num_of_Loan") < 0, None)
             .otherwise(F.col("Num_of_Loan")))
        .withColumn("Num_Credit_Card",
            F.when(F.col("Num_Credit_Card") < 0, None)
             .otherwise(F.col("Num_Credit_Card").cast(IntegerType())))
        .withColumn("Num_Bank_Accounts",
            F.when(F.col("Num_Bank_Accounts") < 0, None)
             .otherwise(F.col("Num_Bank_Accounts").cast(IntegerType())))
        .withColumn("Payment_Behaviour",
            F.when(F.col("Payment_Behaviour").rlike("^[A-Za-z0-9_ ,]+$"), F.col("Payment_Behaviour"))
             .otherwise(None))

    )

    df = (
        df
        .withColumn("Payment_of_Min_Amount",
            F.when(F.col("Payment_of_Min_Amount") == "Yes", 1)
             .when(F.col("Payment_of_Min_Amount") == "No", 0)
             .otherwise(None))
        .withColumn("min_pay_info_missing",
            F.when(F.col("Payment_of_Min_Amount").isin("NM", "", None), 1).otherwise(0))
        .withColumn("credit_history_months", credit_age_to_months("Credit_History_Age"))
        .withColumn("credit_history_years",
            (credit_age_to_months("Credit_History_Age") / 12.0).cast(FloatType()))
        .withColumn("dti",
            F.when(F.col("Annual_Income") > 0,
                   F.col("Outstanding_Debt") / F.col("Annual_Income"))
             .otherwise(None).cast(FloatType()))
        .withColumn("card_num_suspicious",
            F.when(F.col("Num_Credit_Card") > 20, 1).otherwise(0))
        .withColumn("bank_num_suspicious",
            F.when(F.col("Num_Bank_Accounts") > 20, 1).otherwise(0))
    )

    return df


In [42]:
from pathlib import Path
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import IntegerType, FloatType, DateType

spark = SparkSession.builder.appName("silver-test-4-tables").getOrCreate()

BRONZE_ROOT = Path("data_mart/bronze")
SILVER_ROOT = Path("data_mart/silver_test")   
SNAPSHOT = "2024-11-01"

In [43]:
def process_one(table, enrich_fn):
    src = BRONZE_ROOT / table / f"bronze_{table}_{SNAPSHOT.replace('-', '_')}.csv"
    if not src.exists():
        print(f"{table}: file not found → skip")
        return None
    df = spark.read.option("header", "true").csv(str(src))
    df_silver = enrich_fn(df)
    out_path = SILVER_ROOT / table
    out_path.mkdir(parents=True, exist_ok=True)
    df_silver.write.mode("overwrite").parquet(str(out_path / f"silver_{table}_{SNAPSHOT.replace('-', '_')}.parquet"))
    print(f"{table}: {df_silver.count()} rows written")
    return df_silver

TABLES = {
    "lms_loan_daily": augment_lms,
    "feature_clickstream": augment_clickstream,
    "features_attributes": augment_attributes,
    "features_financials": augment_financials,
}

for tbl, fn in TABLES.items():
    process_one(tbl, fn)

for tbl in TABLES.keys():
    path = SILVER_ROOT / tbl / f"silver_{tbl}_{SNAPSHOT.replace('-', '_')}.parquet"
    if path.exists():
        print(f"\nPreview of {tbl}")
        spark.read.parquet(str(path)).show(5, truncate=False)

lms_loan_daily: 5501 rows written
feature_clickstream: 8974 rows written
features_attributes: 488 rows written
features_financials: 488 rows written

Preview of lms_loan_daily
+---------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+-------------+---+-------------------+-----------------+---+--------------+----------------------+
|loan_id              |Customer_ID|loan_start_date|tenure|installment_num|loan_amt|due_amt|paid_amt|overdue_amt|balance|snapshot_date|mob|installments_missed|first_missed_date|dpd|remaining_term|days_since_origination|
+---------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+-------------+---+-------------------+-----------------+---+--------------+----------------------+
|CUS_0x100b_2024_03_01|CUS_0x100b |2024-03-01     |10    |8              |10000   |1000.0 |1000.0  |0.0        |2000.0 |2024-11-01   |8  |0                  |NULL     

In [44]:
from IPython.display import display

TABLES = [
    "lms_loan_daily",
    "feature_clickstream",
    "features_attributes",
    "features_financials",
]

for tbl in TABLES:
    path = f"data_mart/silver_test/{tbl}/silver_{tbl}_{SNAPSHOT.replace('-', '_')}.parquet"
    sdf  = spark.read.parquet(path)
    
    # Take a small sample (e.g., 10 rows) and convert
    pdf  = sdf.limit(20).toPandas()        # <-- key line
    
    print(f"\n=== {tbl}  (total rows: {sdf.count()}) ===")
    display(pdf)                           # pretty, scroll-able view in Jupyter



=== lms_loan_daily  (total rows: 5501) ===


,loan_id,Customer_ID,loan_start_date,tenure,installment_num,loan_amt,due_amt,paid_amt,overdue_amt,balance,snapshot_date,mob,installments_missed,first_missed_date,dpd,remaining_term,days_since_origination
0,CUS_0x100b_2024_03_01,CUS_0x100b,2024-03-01,10,8,10000,1000.0,1000.0,0.0,2000.0,2024-11-01,8,0,None,0,2.0,245
1,CUS_0x102d_2024_01_01,CUS_0x102d,2024-01-01,10,10,10000,1000.0,1000.0,0.0,0.0,2024-11-01,10,0,None,0,0.0,305
2,CUS_0x102e_2024_04_01,CUS_0x102e,2024-04-01,10,7,10000,1000.0,0.0,5000.0,8000.0,2024-11-01,7,5,2024-06-01,153,3.0,214
3,CUS_0x1038_2024_10_01,CUS_0x1038,2024-10-01,10,1,10000,1000.0,1000.0,0.0,9000.0,2024-11-01,1,0,None,0,9.0,31
4,CUS_0x1048_2024_02_01,CUS_0x1048,2024-02-01,10,9,10000,1000.0,0.0,8000.0,9000.0,2024-11-01,9,8,2024-03-01,245,1.0,274
5,CUS_0x104f_2024_10_01,CUS_0x104f,2024-10-01,10,1,10000,1000.0,1000.0,0.0,9000.0,2024-11-01,1,0,None,0,9.0,31
6,CUS_0x1051_2024_01_01,CUS_0x1051,2024-01-01,10,10,10000,1000.0,1000.0,0.0,0.0,2024-11-01,10,0,None,0,0.0,305
7,CUS_0x1075_2024_05_01,CUS_0x1075,2024-05-01,10,6,10000,1000.0,0.0,2000.0,6000.0,2024-11-01,6,2,2024-09-01,61,4.0,184
8,CUS_0x107e_2024_11_01,CUS_0x107e,2024-11-01,10,0,10000,0.0,0.0,0.0,10000.0,2024-11-01,0,0,None,0,10.0,0
9,CUS_0x1087_2024_09_01,CUS_0x1087,2024-09-01,10,2,10000,1000.0,1000.0,0.0,8000.0,2024-11-01,2,0,None,0,8.0,61



=== feature_clickstream  (total rows: 8974) ===


,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date,clickstream_mean
0,287,121,23,237,35,88,180,-4,72,48,...,63,146,-10,44,177,-111,115,CUS_0x1037,2024-11-01,96.20
1,322,92,171,150,29,84,120,60,155,-48,...,90,262,-73,89,-47,112,127,CUS_0x1069,2024-11-01,105.45
2,118,177,66,97,-33,64,-51,108,249,11,...,157,225,171,-99,-103,78,131,CUS_0x114a,2024-11-01,78.80
3,238,256,38,129,60,-2,213,75,215,219,...,227,49,21,-38,40,-12,66,CUS_0x1184,2024-11-01,97.95
4,203,-5,65,128,-4,83,82,137,37,243,...,16,75,36,110,-47,209,54,CUS_0x1297,2024-11-01,86.15
5,294,281,42,104,72,36,15,38,93,-90,...,179,-24,163,59,104,234,63,CUS_0x12fb,2024-11-01,103.60
6,31,15,234,23,242,2,197,180,174,145,...,-65,284,71,241,119,-63,-27,CUS_0x1325,2024-11-01,103.40
7,-6,-19,227,167,22,77,101,368,226,183,...,306,45,122,123,103,108,-13,CUS_0x1341,2024-11-01,123.85
8,-11,180,26,87,241,-1,153,151,184,354,...,77,231,57,134,300,164,65,CUS_0x1375,2024-11-01,129.20
9,86,108,206,93,188,-174,55,-15,64,188,...,91,45,121,107,104,393,-2,CUS_0x13a8,2024-11-01,92.75



=== features_attributes  (total rows: 488) ===


,Customer_ID,Name,Age,SSN,Occupation,snapshot_date,age_band,has_valid_ssn
0,CUS_0x107e,Lianaj,15.0,463-69-6790,Architect,2024-11-01,18-24,1
1,CUS_0x10b3,Johnsona,48.0,293-45-3004,Mechanic,2024-11-01,45-54,1
2,CUS_0x10df,Paritosh Bansalo,27.0,279-20-3097,Musician,2024-11-01,25-34,1
3,CUS_0x10e7,Carewj,NaN,094-89-6380,NA,2024-11-01,55+,1
4,CUS_0x1144,Douwet,42.0,720-70-0565,Doctor,2024-11-01,35-44,1
5,CUS_0x1152,Gellerv,29.0,330-02-2598,Accountant,2024-11-01,25-34,1
6,CUS_0x11ae,Xiaoyi Shaou,22.0,032-97-5902,Architect,2024-11-01,18-24,1
7,CUS_0x1231,James Reganc,39.0,780-20-7811,Journalist,2024-11-01,35-44,1
8,CUS_0x126d,Brownj,36.0,047-21-1235,Architect,2024-11-01,35-44,1
9,CUS_0x127d,McCoolg,43.0,829-08-2773,Doctor,2024-11-01,35-44,1



=== features_financials  (total rows: 488) ===


,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,snapshot_date,min_pay_info_missing,credit_history_months,credit_history_years,dti,card_num_suspicious,bank_num_suspicious
0,CUS_0x107e,75095.906250,6269.9925,5,6,15,6.0,"Personal Loan, Payday Loan, Personal Loan, Stu...",9,10,...,10000.000000,Low_spent_Medium_value_payments,234.06656029063686,2024-11-01,0,144,12.000000,0.013179,0,0
1,CUS_0x10b3,136150.593750,11127.883333333331,1,5,9,3.0,"Student Loan, Student Loan, and Auto Loan",10,3,...,1302.357422,Low_spent_Small_value_payments,898.3588009083421,2024-11-01,0,328,27.333334,0.001912,0,0
2,CUS_0x10df,9929.235352,828.4362500000002,4,8,23,3.0,"Not Specified, Auto Loan, and Student Loan",16,18,...,74.964432,Low_spent_Small_value_payments,260.9159334781246,2024-11-01,0,86,7.166667,0.234467,0,0
3,CUS_0x10e7,64933.761719,5368.1466666666665,3,3,33,NaN,"Mortgage Loan, Auto Loan, Home Equity Loan, an...",15,19,...,186.146667,High_spent_Medium_value_payments,406.9754192189962,2024-11-01,0,172,14.333333,0.041568,0,0
4,CUS_0x1144,135851.234375,11418.936666666665,4,4,18,2.0,"Home Equity Loan, and Auto Loan",28,19,...,1139.083740,Low_spent_Medium_value_payments,76.25273594871554,2024-11-01,0,302,25.166666,0.005117,0,0
5,CUS_0x1152,31316.470703,2510.705833333333,0,2,6,2.0,"Payday Loan, and Personal Loan",13,5,...,124.976791,High_spent_Small_value_payments,355.2718152011777,2024-11-01,0,222,18.500000,0.025574,0,0
6,CUS_0x11ae,14550.519531,1392.543333333333,7,10,25,9.0,"Personal Loan, Not Specified, Home Equity Loan...",43,20,...,123.138710,Low_spent_Small_value_payments,210.2045926714747,2024-11-01,0,132,11.000000,0.305487,0,0
7,CUS_0x1231,30739.300781,2298.608333333333,4,4,13,6.0,"Student Loan, Personal Loan, Payday Loan, Payd...",15,8,...,106.453262,High_spent_Small_value_payments,248.16942817813796,2024-11-01,0,218,18.166666,0.044425,0,0
8,CUS_0x126d,34710.601562,3170.55,6,3,20,NaN,"Credit-Builder Loan, Payday Loan, Credit-Build...",22,10,...,324.462402,Low_spent_Small_value_payments,85.48451747279788,2024-11-01,0,180,15.000000,0.042135,0,0
9,CUS_0x127d,15096.849609,1033.0708333333334,8,6,20,4.0,"Home Equity Loan, Home Equity Loan, Home Equit...",18,20,...,63.668598,High_spent_Small_value_payments,264.3249250589178,2024-11-01,0,207,17.250000,0.127711,0,0


In [45]:
import shutil
import os

def delete_folder(folder_path: str):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    else:
        print(f"Folder does not exist: {folder_path}")

delete_folder("data_mart/silver_test")

Deleted folder: data_mart/silver_test
